# Project 5d-SIR models in SPACE

Until now, we have been approaching disease models in a way that lumps spatial information: entire population classes, despite the fact that humans are geographically separated, are treated as single variables (S,I, or R).  However, it is possible to simulate disease propagation at rather different levels of detail.  One very interesting and very detailed way is to use agent-based models, which means that every single person in the environment is treated individually.  Such models are quite expensive to run.  Another way is a lattice SIR model, which is what we will explore in this section (partially as a means of motivating our next class of model).  

The lattice model is very simple.  Instead of modelling an entire population via a single S, I, and R, we will define an $n\times n$ grid and each location on the grid will have its own SIR status (to be concrete, you could represent these by three numpy arrays).  At each of these locations $k$, we might have dynamics like
$$
\frac{\partial S_{k}}{\partial t} = -\beta S_{k} I_{k}
$$
$$
\frac{\partial I_{k}}{\partial t} = \beta S_{k} I_{k} - \gamma I_{k}
$$
$$
\frac{\partial R_{k}}{\partial t} = \gamma I_{k}.
$$
However, this wouldn't be a very interesting model because each of the locations on the grid are independent from one another.  Instead, we would like adjacent grid cells to be *coupled* (this is similar to the Game of Life).  What sort of coupling makes sense?  There are many possible situations that we could model now that we have some spatial dependency.  One simple possibility states that: rather than susceptible individuals being converted into infected individuals as $-\beta S_{k} I_{k}$, we could have that  
$$
\frac{\partial S_{k}}{\partial t} = -\beta S_{k}\; \sum_{j\in adj(k)} \frac{I_j}{9},
$$
(with a similar modification to the rate of change in infected) which is to say that the infection probability is now proportional to the average number of infected individuals in the cells adjacent to $k$. 

**Implement and run the model described above. Begin by using $n=50$, $\beta=1$, and $\gamma=0.1$.  Assume periodic boundary conditions.  Start with only a single grid cell somewhere in the domain containing an infected individual.**  A few hints for modifying your code: just like with the molecular dynamics problem, you'll need to figure out a way to map from a vector of variables of size $3N$, with $N=n^2$ to the S, I, and R values at each location in a grid.  Internally within my rhs function I (for example) took the first $N$ elements in my vector to be my $S$ values and then explicitly reshaped them back into a grid using np.reshape.  At the end of this function, I turned grid variables back into a vector using np.ravel and concatenation.  For computing the spatial average, I suggest the convolve2d function from scipy.signal (pay attention to the keyword arguments for easy implementation of boundaries, in particular boundary='wrap').

In [103]:
n = 50
beta = 1
gamma = 0.1

S = np.full((n,n), 1)
I = np.full((n,n,), 0)
R = np.full((n,n,), 1)

I[np.random.choice(list(range(50))), np.random.choice(list(range(50)))] = 1

In [104]:
from scipy.signal import convolve2d

K_array = [1/9, 1/9, 1/9]
K = [K_array, K_array, K_array] # convul matrix

new_I = convolve2d(I,K, mode='same')
new_I.shape



(50, 50)

In [105]:
import numpy as np
import ode_methods as om
import matplotlib.pyplot as plt

class SIR:
    
    def __init__(self,beta=10,gamma=0.1):
        
        self.beta = beta    # Infection rate
        self.gamma = gamma  # Recovery/death rate
        self.n_dof = 3      
        
    def rhs(self,t,u):
        S = u[0]
        I = u[1]
        R = u[2]

        I = convolve2d(I,K, mode='same')

        dSdt = -self.beta * S * I
        dIdt = self.beta * S * I - self.gamma * I
        dRdt = self.gamma * I

        dudt = 0
        dudt = np.array([dSdt, dIdt, dRdt])

        '''
        # the right hand side of the ode (or $\mathcal{F}(t,u)$)
        dudt = np.array([-self.beta*u[0]*u[1],
                         self.beta*u[0]*u[1] - self.gamma*u[1],
                         self.gamma*u[1]])  
        '''
        return dudt
    
method = om.Midpoint()

#u0 = np.array([1,0.1,0])
u0 = np.array([S,I,R])

s = SIR(beta, gamma)
integrator = om.Integrator(s,method)
t,u = integrator.integrate([0,100],0.01,u0)



C:\Users\Derrick\AppData\Local\Temp\ipykernel_14028\2885030788.py:20: RuntimeWarning: overflow encountered in multiply
  dSdt = -self.beta * S * I
C:\Users\Derrick\AppData\Local\Temp\ipykernel_14028\2885030788.py:21: RuntimeWarning: overflow encountered in multiply
  dIdt = self.beta * S * I - self.gamma * I
C:\Users\Derrick\AppData\Local\Temp\ipykernel_14028\2885030788.py:21: RuntimeWarning: invalid value encountered in subtract
  dIdt = self.beta * S * I - self.gamma * I


In [100]:
u.sum()

10000.99999999997

**Animate the solution (I suggest repurposing code from Conway's Game of Life).  Describe the patterns that you see of the population classes in space and time.** 

### Barriers
We can utilize this model to explore the influence of geographic barriers to disease propagation.  First, modify your code to accept a spatially varying value of $\beta$, i.e. 
$$
\frac{\partial S_{k}}{\partial t} = -\beta_k S_{k}\; \sum_{j\in adj(k)} \frac{I_j}{9}.
$$
Second, modify your boundary conditions to *not* be periodic.  If you used scipy.convolve, then you can do this by changing the boundary keyword to 'symm'.    

**Now, create a simulation that utilizes a barrier.  For example, you might set the middle 1/3 of the domain to have $\beta=0$.  How does the simulation change?  If the barrier is imperfect, does the disease still reach the other side?**